<a href="https://colab.research.google.com/github/Glaydyesther/hacking-civico/blob/master/tareas/03_exploracion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
# Para descargar el conjunto de datos
!wget http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

# Para descargar el diccionario de los datos
!wget http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip

# Descomprimimos ambas carpetas y movemos archivos
!unzip datos_abiertos_covid19.zip
!unzip diccionario_datos_covid19.zip
!mv ./diccionario_datos_covid19/*.xlsx .
!mv *COVID19MEXICO.csv COVID19MEXICO.csv

# Eliminamos las carpetas comprimidas
!rm -rf diccionario_datos_covid19
!rm datos_abiertos_covid19.zip
!rm diccionario_datos_covid19.zip

--2020-09-13 18:55:44--  http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 187.210.186.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|187.210.186.146|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31945189 (30M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip’

datos_abiertos_covi 100%[===================>]  30.46M  7.04MB/s    in 6.8s    

2020-09-13 18:55:51 (4.50 MB/s) - ‘datos_abiertos_covid19.zip’ saved [31945189/31945189]

--2020-09-13 18:55:51--  http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip
Resolving epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)... 187.191.75.207
Connecting to epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)|187.191.75.207|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ep

In [2]:
import io
import statistics
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
path='/content/drive/My Drive/datos_abiertos_covid19.zip'
data = pd.read_csv(path, encoding='latin-1')

In [6]:
data.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-09-11,1779cd,2,3,15,2,15,15,106,2,2020-05-27,2020-05-25,9999-99-99,2,1,58,1,97,2,1,2,2,2,1,2,2,2,2,2,2,1,99,MÃ©xico,99,2
1,2020-09-11,05a196,2,4,12,2,20,12,29,2,2020-03-30,2020-03-30,2020-04-06,1,1,36,1,97,2,2,2,2,2,2,2,2,1,2,2,99,1,99,MÃ©xico,99,2
2,2020-09-11,08f0e2,2,4,15,1,15,15,33,1,2020-04-12,2020-04-11,9999-99-99,97,2,60,1,2,2,2,2,2,2,2,2,2,1,2,1,99,1,99,MÃ©xico,99,97
3,2020-09-11,100180,2,4,9,1,10,9,15,1,2020-05-06,2020-05-02,9999-99-99,97,2,30,1,2,2,2,2,2,2,1,2,2,2,2,2,99,1,99,MÃ©xico,99,97
4,2020-09-11,1622ec,2,4,15,1,15,15,109,2,2020-05-10,2020-05-10,2020-05-27,2,2,87,1,2,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,2


In [7]:
path_dic='/content/drive/My Drive/Descriptores_0419.xlsx'
diccionario =pd.read_excel(path_dic)

In [8]:
diccionario

,Nº,NOMBRE DE VARIABLE,DESCRIPCIÓN DE VARIABLE,FORMATO O FUENTE
0,1,FECHA_ACTUALIZACION,"La base de datos se alimenta diariamente, esta...",AAAA-MM-DD
1,2,ID_REGISTRO,Número identificador del caso,TEXTO
2,3,ORIGEN,La vigilancia centinela se realiza a través de...,CATÁLOGO: ORIGEN ...
3,4,SECTOR,Identifica el tipo de institución del Sistema ...,CATÁLOGO: SECTOR ...
4,5,ENTIDAD_UM,Identifica la entidad donde se ubica la unidad...,CATALÓGO: ENTIDADES
5,6,SEXO,Identifica al sexo del paciente.,CATÁLOGO: SEXO
6,7,ENTIDAD_NAC,Identifica la entidad de nacimiento del paciente.,CATALÓGO: ENTIDADES
7,8,ENTIDAD_RES,Identifica la entidad de residencia del paciente.,CATALÓGO: ENTIDADES
8,9,MUNICIPIO_RES,Identifica el municipio de residencia del paci...,CATALÓGO: MUNICIPIOS
9,10,TIPO_PACIENTE,Identifica el tipo de atención que recibió el ...,CATÁLOGO: TIPO_PACIENTE


In [11]:
path_cat = '/content/drive/My Drive/Catalogos_0412.xlsx'
sheets = ['ORIGEN', 'SECTOR', 'SEXO', 'TIPO_PACIENTE', 'SI_NO', 'NACIONALIDAD', 'RESULTADO', 'de ENTIDADES', 'MUNICIPIOS']
catalogues = {}
for sheet in sheets:
  catalogue = pd.read_excel(path_cat, sheet_name='Catálogo ' + sheet)
  catalogue.columns = catalogue.columns.str.replace(' ', '_')
  catalogue.columns = catalogue.columns.str.replace('.', '')
  catalogue.columns = catalogue.columns.str.lower()
  new_catalogue = {sheet: catalogue}
  catalogues.update(new_catalogue)

In [12]:
catalogues['SEXO']

,clave,descripción
0,1,MUJER
1,2,HOMBRE
2,99,NO ESPECIFICADO


In [13]:
print('Número de filas: {}'.format(data.shape[0]))
print('Número de columnas: {}'.format(data.shape[1]))

Número de filas: 1496322
Número de columnas: 35


In [14]:
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [15]:
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('.', '')
data.columns = data.columns.str.lower()

In [16]:
for col in data.columns:
  print('Valores nulos en "{}": {}'.format(col,data[col].isna().sum()))

Valores nulos en "fecha_actualizacion": 0
Valores nulos en "id_registro": 0
Valores nulos en "origen": 0
Valores nulos en "sector": 0
Valores nulos en "entidad_um": 0
Valores nulos en "sexo": 0
Valores nulos en "entidad_nac": 0
Valores nulos en "entidad_res": 0
Valores nulos en "municipio_res": 0
Valores nulos en "tipo_paciente": 0
Valores nulos en "fecha_ingreso": 0
Valores nulos en "fecha_sintomas": 0
Valores nulos en "fecha_def": 0
Valores nulos en "intubado": 0
Valores nulos en "neumonia": 0
Valores nulos en "edad": 0
Valores nulos en "nacionalidad": 0
Valores nulos en "embarazo": 0
Valores nulos en "habla_lengua_indig": 0
Valores nulos en "diabetes": 0
Valores nulos en "epoc": 0
Valores nulos en "asma": 0
Valores nulos en "inmusupr": 0
Valores nulos en "hipertension": 0
Valores nulos en "otra_com": 0
Valores nulos en "cardiovascular": 0
Valores nulos en "obesidad": 0
Valores nulos en "renal_cronica": 0
Valores nulos en "tabaquismo": 0
Valores nulos en "otro_caso": 0
Valores nulos 

In [17]:
data.dtypes

fecha_actualizacion    object
id_registro            object
origen                  int64
sector                  int64
entidad_um              int64
sexo                    int64
entidad_nac             int64
entidad_res             int64
municipio_res           int64
tipo_paciente           int64
fecha_ingreso          object
fecha_sintomas         object
fecha_def              object
intubado                int64
neumonia                int64
edad                    int64
nacionalidad            int64
embarazo                int64
habla_lengua_indig      int64
diabetes                int64
epoc                    int64
asma                    int64
inmusupr                int64
hipertension            int64
otra_com                int64
cardiovascular          int64
obesidad                int64
renal_cronica           int64
tabaquismo              int64
otro_caso               int64
resultado               int64
migrante                int64
pais_nacionalidad      object
pais_orige

In [18]:
def cast_datetime_cols(df):
    timestamp_cols = [col for col in df.columns if "fecha" in col]
    df[timestamp_cols] = df[timestamp_cols].apply(lambda date_col: pd.to_datetime(date_col, errors="coerce"), axis=0)
    return df

data = cast_datetime_cols(data)

In [19]:
data.head()

,fecha_actualizacion,id_registro,origen,sector,entidad_um,sexo,entidad_nac,entidad_res,municipio_res,tipo_paciente,fecha_ingreso,fecha_sintomas,fecha_def,intubado,neumonia,edad,nacionalidad,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,resultado,migrante,pais_nacionalidad,pais_origen,uci
0,2020-09-11,1779cd,2,3,15,2,15,15,106,2,2020-05-27,2020-05-25,NaT,2,1,58,1,97,2,1,2,2,2,1,2,2,2,2,2,2,1,99,MÃ©xico,99,2
1,2020-09-11,05a196,2,4,12,2,20,12,29,2,2020-03-30,2020-03-30,2020-04-06,1,1,36,1,97,2,2,2,2,2,2,2,2,1,2,2,99,1,99,MÃ©xico,99,2
2,2020-09-11,08f0e2,2,4,15,1,15,15,33,1,2020-04-12,2020-04-11,NaT,97,2,60,1,2,2,2,2,2,2,2,2,2,1,2,1,99,1,99,MÃ©xico,99,97
3,2020-09-11,100180,2,4,9,1,10,9,15,1,2020-05-06,2020-05-02,NaT,97,2,30,1,2,2,2,2,2,2,1,2,2,2,2,2,99,1,99,MÃ©xico,99,97
4,2020-09-11,1622ec,2,4,15,1,15,15,109,2,2020-05-10,2020-05-10,2020-05-27,2,2,87,1,2,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,2


In [20]:
data = (
    # unimos con el catálogo MUNICIPIOS y ENTIDADES
    data.merge(catalogues['de ENTIDADES'], how='left', left_on='entidad_res', right_on='clave_entidad').
    drop(columns=['entidad_res', 'abreviatura']).
    rename(columns={'entidad_federativa':'entidad_res'}).
    merge(catalogues['MUNICIPIOS'], how='left', left_on=['clave_entidad','municipio_res'], right_on=['clave_entidad','clave_municipio']).
    drop(columns=['municipio_res', 'clave_entidad', 'clave_municipio']).
    rename(columns={'municipio':'municipio_res'})
)

In [21]:
entidades_cols = [col for col in data.columns if "entidad" in col and col != 'entidad_res']
for col in entidades_cols:
    data = (
        data.merge(catalogues['de ENTIDADES'], how='left', left_on=col, right_on='clave_entidad').
        drop(columns=[col, 'abreviatura', 'clave_entidad']).
        rename(columns={'entidad_federativa':col})
    )

In [22]:
sino_cols = ['intubado', 'neumonia', 'embarazo', 'habla_lengua_indig', 'diabetes', 'epoc', 'asma', 'inmusupr',
             'hipertension', 'otra_com', 'cardiovascular', 'obesidad', 'renal_cronica', 'tabaquismo', 
             'otro_caso', 'migrante', 'uci']
for col in sino_cols:
    data = (
        data.merge(catalogues['SI_NO'], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [51]:
catalogues['RESULTADO'].columns

Index(['clave', 'descripción'], dtype='object')

In [52]:
catalogues['RESULTADO'].columns = ['clave', 'descripción']
catalogues['RESULTADO'].columns

Index(['clave', 'descripción'], dtype='object')

In [61]:
catalogues['RESULTADO']=catalogues['RESULTADO'].drop(index=1)

In [62]:
catalogues['RESULTADO']['clave'] = catalogues['RESULTADO']['clave'].astype(int)

In [74]:
data.head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad
0,2020-09-11,1779cd,2020-05-27,2020-05-25,NaT,58,1,MÃ©xico,99,MÉXICO,TOLUCA,MÉXICO,MÉXICO,NO,SI,NO APLICA,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ESTATAL,HOMBRE,HOSPITALIZADO,MEXICANA
1,2020-09-11,05a196,2020-03-30,2020-03-30,2020-04-06,36,1,MÃ©xico,99,GUERRERO,CHILPANCINGO DE LOS BRAVO,GUERRERO,OAXACA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA
2,2020-09-11,08f0e2,2020-04-12,2020-04-11,NaT,60,1,MÃ©xico,99,MÉXICO,ECATEPEC DE MORELOS,MÉXICO,MÉXICO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,SI,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA
3,2020-09-11,100180,2020-05-06,2020-05-02,NaT,30,1,MÃ©xico,99,CIUDAD DE MÉXICO,CUAUHTÉMOC,CIUDAD DE MÉXICO,DURANGO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA
4,2020-09-11,1622ec,2020-05-10,2020-05-10,2020-05-27,87,1,MÃ©xico,99,MÉXICO,TULTITLÁN,MÉXICO,MÉXICO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA


In [75]:
data.query('nacionalidad== "MEXICANA"').head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad
0,2020-09-11,1779cd,2020-05-27,2020-05-25,NaT,58,1,MÃ©xico,99,MÉXICO,TOLUCA,MÉXICO,MÉXICO,NO,SI,NO APLICA,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ESTATAL,HOMBRE,HOSPITALIZADO,MEXICANA
1,2020-09-11,05a196,2020-03-30,2020-03-30,2020-04-06,36,1,MÃ©xico,99,GUERRERO,CHILPANCINGO DE LOS BRAVO,GUERRERO,OAXACA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA


In [78]:
data.query('nacionalidad=="MEXICANA"& entidad_res=="SINALOA"').head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad
11,2020-09-11,1413b4,2020-04-29,2020-04-24,NaT,38,1,MÃ©xico,99,SINALOA,CULIACÁN,SINALOA,SINALOA,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA
22,2020-09-11,1e6e4c,2020-04-20,2020-04-17,NaT,31,1,MÃ©xico,99,SINALOA,GUASAVE,SINALOA,SINALOA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA


In [79]:
data.groupby('sexo').edad.mean()

sexo
HOMBRE    42.685437
MUJER     41.653360
Name: edad, dtype: float64

In [80]:
data.groupby('sexo').agg({'edad':['min','mean', 'max']})

edad                
        min       mean  max
sexo                       
HOMBRE    0  42.685437  120
MUJER     0  41.653360  120

In [82]:
data.query('nacionalidad=="MEXICANA"').head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad
0,2020-09-11,1779cd,2020-05-27,2020-05-25,NaT,58,1,MÃ©xico,99,MÉXICO,TOLUCA,MÉXICO,MÉXICO,NO,SI,NO APLICA,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ESTATAL,HOMBRE,HOSPITALIZADO,MEXICANA
1,2020-09-11,05a196,2020-03-30,2020-03-30,2020-04-06,36,1,MÃ©xico,99,GUERRERO,CHILPANCINGO DE LOS BRAVO,GUERRERO,OAXACA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA
2,2020-09-11,08f0e2,2020-04-12,2020-04-11,NaT,60,1,MÃ©xico,99,MÉXICO,ECATEPEC DE MORELOS,MÉXICO,MÉXICO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,SI,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA
3,2020-09-11,100180,2020-05-06,2020-05-02,NaT,30,1,MÃ©xico,99,CIUDAD DE MÉXICO,CUAUHTÉMOC,CIUDAD DE MÉXICO,DURANGO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA
4,2020-09-11,1622ec,2020-05-10,2020-05-10,2020-05-27,87,1,MÃ©xico,99,MÉXICO,TULTITLÁN,MÉXICO,MÉXICO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA


In [83]:
data['defuncion'] = ~data.fecha_def.isna()
data['mes_def'] = data.fecha_def.dt.month
data['año_def'] = data.fecha_def.dt.year
data['mes_ingreso'] = data.fecha_ingreso.dt.month
data['año_ingreso'] = data.fecha_ingreso.dt.year

In [84]:
data.head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-11,1779cd,2020-05-27,2020-05-25,NaT,58,1,MÃ©xico,99,MÉXICO,TOLUCA,MÉXICO,MÉXICO,NO,SI,NO APLICA,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ESTATAL,HOMBRE,HOSPITALIZADO,MEXICANA,False,NaN,NaN,5,2020
1,2020-09-11,05a196,2020-03-30,2020-03-30,2020-04-06,36,1,MÃ©xico,99,GUERRERO,CHILPANCINGO DE LOS BRAVO,GUERRERO,OAXACA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,True,4.0,2020.0,3,2020
2,2020-09-11,08f0e2,2020-04-12,2020-04-11,NaT,60,1,MÃ©xico,99,MÉXICO,ECATEPEC DE MORELOS,MÉXICO,MÉXICO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,SI,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020
3,2020-09-11,100180,2020-05-06,2020-05-02,NaT,30,1,MÃ©xico,99,CIUDAD DE MÉXICO,CUAUHTÉMOC,CIUDAD DE MÉXICO,DURANGO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,5,2020
4,2020-09-11,1622ec,2020-05-10,2020-05-10,2020-05-27,87,1,MÃ©xico,99,MÉXICO,TULTITLÁN,MÉXICO,MÉXICO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,True,5.0,2020.0,5,2020


In [85]:
numerical = data.select_dtypes(include='number')
categorical = data.select_dtypes(include=['object', 'bool', 'category', 'datetime64[ns]'])

In [86]:
categorical.head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion
0,2020-09-11,1779cd,2020-05-27,2020-05-25,NaT,MÃ©xico,99,MÉXICO,TOLUCA,MÉXICO,MÉXICO,NO,SI,NO APLICA,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ESTATAL,HOMBRE,HOSPITALIZADO,MEXICANA,False
1,2020-09-11,05a196,2020-03-30,2020-03-30,2020-04-06,MÃ©xico,99,GUERRERO,CHILPANCINGO DE LOS BRAVO,GUERRERO,OAXACA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,True
2,2020-09-11,08f0e2,2020-04-12,2020-04-11,NaT,MÃ©xico,99,MÉXICO,ECATEPEC DE MORELOS,MÉXICO,MÉXICO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,SI,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False
3,2020-09-11,100180,2020-05-06,2020-05-02,NaT,MÃ©xico,99,CIUDAD DE MÉXICO,CUAUHTÉMOC,CIUDAD DE MÉXICO,DURANGO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False
4,2020-09-11,1622ec,2020-05-10,2020-05-10,2020-05-27,MÃ©xico,99,MÉXICO,TULTITLÁN,MÉXICO,MÉXICO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,True


In [87]:
for col in categorical.columns:
  print('Valores únicos en "{}": {}'.format(col,categorical[col].nunique()))

Valores únicos en "fecha_actualizacion": 1
Valores únicos en "id_registro": 1496322
Valores únicos en "fecha_ingreso": 255
Valores únicos en "fecha_sintomas": 255
Valores únicos en "fecha_def": 200
Valores únicos en "pais_nacionalidad": 119
Valores únicos en "pais_origen": 77
Valores únicos en "entidad_res": 32
Valores únicos en "municipio_res": 2163
Valores únicos en "entidad_um": 32
Valores únicos en "entidad_nac": 33
Valores únicos en "intubado": 4
Valores únicos en "neumonia": 3
Valores únicos en "embarazo": 4
Valores únicos en "habla_lengua_indig": 3
Valores únicos en "diabetes": 3
Valores únicos en "epoc": 3
Valores únicos en "asma": 3
Valores únicos en "inmusupr": 3
Valores únicos en "hipertension": 3
Valores únicos en "otra_com": 3
Valores únicos en "cardiovascular": 3
Valores únicos en "obesidad": 3
Valores únicos en "renal_cronica": 3
Valores únicos en "tabaquismo": 3
Valores únicos en "otro_caso": 3
Valores únicos en "migrante": 3
Valores únicos en "uci": 4
Valores únicos en

In [88]:
total = data.shape[0]
for col in categorical.columns:
  print('Porcentaje de valores únicos en "{}": {}%'.format(col,round(categorical[col].nunique()/total*100,2)))

Porcentaje de valores únicos en "fecha_actualizacion": 0.0%
Porcentaje de valores únicos en "id_registro": 100.0%
Porcentaje de valores únicos en "fecha_ingreso": 0.02%
Porcentaje de valores únicos en "fecha_sintomas": 0.02%
Porcentaje de valores únicos en "fecha_def": 0.01%
Porcentaje de valores únicos en "pais_nacionalidad": 0.01%
Porcentaje de valores únicos en "pais_origen": 0.01%
Porcentaje de valores únicos en "entidad_res": 0.0%
Porcentaje de valores únicos en "municipio_res": 0.14%
Porcentaje de valores únicos en "entidad_um": 0.0%
Porcentaje de valores únicos en "entidad_nac": 0.0%
Porcentaje de valores únicos en "intubado": 0.0%
Porcentaje de valores únicos en "neumonia": 0.0%
Porcentaje de valores únicos en "embarazo": 0.0%
Porcentaje de valores únicos en "habla_lengua_indig": 0.0%
Porcentaje de valores únicos en "diabetes": 0.0%
Porcentaje de valores únicos en "epoc": 0.0%
Porcentaje de valores únicos en "asma": 0.0%
Porcentaje de valores únicos en "inmusupr": 0.0%
Porcenta

In [89]:
for col in categorical.drop(columns='id_registro').columns:
  print('Valor más popular en "{}": {}'.format(col, statistics.mode(categorical[col])))

Valor más popular en "fecha_actualizacion": 2020-09-11 00:00:00
Valor más popular en "fecha_ingreso": 2020-07-20 00:00:00
Valor más popular en "fecha_sintomas": 2020-07-20 00:00:00
Valor más popular en "fecha_def": NaT
Valor más popular en "pais_nacionalidad": MÃ©xico
Valor más popular en "pais_origen": 99
Valor más popular en "entidad_res": CIUDAD DE MÉXICO
Valor más popular en "municipio_res": IZTAPALAPA
Valor más popular en "entidad_um": CIUDAD DE MÉXICO
Valor más popular en "entidad_nac": CIUDAD DE MÉXICO
Valor más popular en "intubado": NO APLICA
Valor más popular en "neumonia": NO 
Valor más popular en "embarazo": NO 
Valor más popular en "habla_lengua_indig": NO 
Valor más popular en "diabetes": NO 
Valor más popular en "epoc": NO 
Valor más popular en "asma": NO 
Valor más popular en "inmusupr": NO 
Valor más popular en "hipertension": NO 
Valor más popular en "otra_com": NO 
Valor más popular en "cardiovascular": NO 
Valor más popular en "obesidad": NO 
Valor más popular en "r

In [127]:
data.groupby('hipertension').defuncion.value_counts(normalize=True)

hipertension  defuncion
NO            False        0.960621
              True         0.039379
SE IGNORA     False        0.871967
              True         0.128033
SI            False        0.834159
              True         0.165841
Name: defuncion, dtype: float64

In [132]:
data[data.defuncion].entidad_res.value_counts().sort_index()

AGUASCALIENTES                       596
BAJA CALIFORNIA                     4072
BAJA CALIFORNIA SUR                  526
CAMPECHE                             958
CHIAPAS                             1219
CHIHUAHUA                           2029
CIUDAD DE MÉXICO                   11963
COAHUILA DE ZARAGOZA                2169
COLIMA                               594
DURANGO                              714
GUANAJUATO                          3153
GUERRERO                            2291
HIDALGO                             2069
JALISCO                             3872
MICHOACÁN DE OCAMPO                 1908
MORELOS                             1250
MÉXICO                             13561
NAYARIT                              820
NUEVO LEÓN                          3517
OAXACA                              1621
PUEBLA                              4621
QUERÉTARO                            993
QUINTANA ROO                        1911
SAN LUIS POTOSÍ                     1748
SINALOA         

In [136]:
data.entidad_res.value_counts()

CIUDAD DE MÉXICO                   309595
MÉXICO                             183480
GUANAJUATO                          85362
NUEVO LEÓN                          77193
PUEBLA                              62822
TAMAULIPAS                          61028
TABASCO                             59525
JALISCO                             58174
COAHUILA DE ZARAGOZA                58144
VERACRUZ DE IGNACIO DE LA LLAVE     48678
SAN LUIS POTOSÍ                     45198
MICHOACÁN DE OCAMPO                 43542
SONORA                              42476
YUCATÁN                             31251
SINALOA                             31147
BAJA CALIFORNIA                     29760
GUERRERO                            28678
DURANGO                             22199
OAXACA                              21896
CHIHUAHUA                           21204
BAJA CALIFORNIA SUR                 19851
HIDALGO                             19816
AGUASCALIENTES                      19519
TLAXCALA                          

In [142]:
data[data.defuncion].tipo_paciente.value_counts().sort_index()

AMBULATORIO       9558
HOSPITALIZADO    80096
Name: tipo_paciente, dtype: int64